## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-01-18-59-04 +0000,nyt,"To Avoid Pharma Tariffs, U.K. Agrees to Trump’...",https://www.nytimes.com/2025/12/01/business/us...
1,2025-12-01-18-55-20 +0000,bbc,OBR head resigns over Budget day publishing error,https://www.bbc.com/news/articles/cd74v35p77jo...
2,2025-12-01-18-53-34 +0000,nyt,Zelensky Turns to Europe After Talks With U.S....,https://www.nytimes.com/2025/12/01/world/europ...
3,2025-12-01-18-53-03 +0000,nypost,Starbucks to pay $35M to NYC workers to settle...,https://nypost.com/2025/12/01/business/starbuc...
4,2025-12-01-18-52-32 +0000,nyt,Winter Storm Expected to Slam Northeast U.S. W...,https://www.nytimes.com/2025/12/01/us/snowstor...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,38
391,new,13
190,up,13
83,court,9
158,house,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
135,2025-12-01-11-00-00 +0000,latimes,"Attacks on ICE up 1,000%? Trump administration...",https://www.latimes.com/california/story/2025-...,86
87,2025-12-01-14-38-59 +0000,nypost,Trump backs Hegseth after report claims war se...,https://nypost.com/2025/12/01/us-news/trump-ba...,83
85,2025-12-01-14-42-17 +0000,nypost,Appeals court disqualifies former Trump lawyer...,https://nypost.com/2025/12/01/us-news/appeals-...,81
53,2025-12-01-16-39-09 +0000,bbc,Appeals court disqualifies ex-Trump lawyer Ali...,https://www.bbc.com/news/articles/cewjg272dq9o...,81
57,2025-12-01-16-20-00 +0000,wsj,Court Upholds Disqualification of Trump’s Top ...,https://www.wsj.com/us-news/law/court-upholds-...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
135,86,2025-12-01-11-00-00 +0000,latimes,"Attacks on ICE up 1,000%? Trump administration...",https://www.latimes.com/california/story/2025-...
207,47,2025-11-30-23-47-00 +0000,wsj,Congress launched inquiries and lawmakers from...,https://www.wsj.com/politics/national-security...
113,43,2025-12-01-12-41-27 +0000,bbc,Three children among four killed in shooting a...,https://www.bbc.com/news/articles/cvg81p61dnmo...
156,41,2025-12-01-09-57-37 +0000,cbc,"Death toll from floods in Indonesia, Sri Lanka...",https://www.cbc.ca/news/world/death-toll-from-...
244,37,2025-11-30-19-34-17 +0000,nypost,White House names and shames ‘biased’ news out...,https://nypost.com/2025/11/30/us-news/white-ho...
3,36,2025-12-01-18-53-03 +0000,nypost,Starbucks to pay $35M to NYC workers to settle...,https://nypost.com/2025/12/01/business/starbuc...
112,34,2025-12-01-12-43-00 +0000,wsj,Hong Kong’s Response to Deadly Fire Is Squeeze...,https://www.wsj.com/world/china/hong-kongs-res...
199,32,2025-12-01-01-00-00 +0000,wsj,"More than other generations, 20-somethings are...",https://www.wsj.com/economy/consumers/gen-z-sh...
246,29,2025-11-30-19-28-51 +0000,nypost,Marco Rubio insists peace deal must ensure Ukr...,https://nypost.com/2025/11/30/us-news/ukraine-...
107,28,2025-12-01-12-59-45 +0000,nypost,Venezuela’s Maduro spotted for first time in d...,https://nypost.com/2025/12/01/us-news/venezuel...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
